На основе записи диалога Клиента и Менеджера, используя материалы занятия, проанализируйте предложенные тарифы, пуш-уведомления и варианты оплаты, а затем составьте итоговый отчет на основе проведенного анализа. Ссылка на запись диалога Менеджера и Клиента:  https://disk.yandex.ru/d/IqOQeMnvd6YSKg

In [2]:
#@title Установка библиотек
!pip install -q openai==1.61.1 faiss-cpu==1.10.0 langchain==0.3.20 langchain-core==0.3.41 langchain-openai==0.3.7 langchain_community==0.3.19 tiktoken==0.9.0

In [15]:
#@title Импорт модулей
import os
from google.colab import userdata
from langchain.text_splitter import NLTKTextSplitter
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
import tiktoken
from IPython.display import HTML, display
from google.colab import output
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from openai import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from pprint import pprint
import re
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [11]:
#@title Чтение ключа OPENAI_API_KEY
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
#client = OpenAI()
BASE_URL = "https://openrouter.ai/api/v1"
API_KEY = os.environ.get("OPENAI_API_KEY")
MODEL = "deepseek/deepseek-r1-distill-qwen-32b:free"
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

In [16]:
#@title Объявляем общие функции

# определение класса bcolors в Python, который используется для добавления цвета и стилей к тексту, выводимому в терминале.
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# Функция load_document_text принимает один параметр file_path, который представляет собой путь к файлу.
# Она открывает файл в режиме чтения с указанием кодировки 'utf-8', читает его содержимое полностью в переменную text, а затем закрывает файл.
# Функция возвращает прочитанное содержимое файла в виде строки.
def load_document_text(file_path) -> str:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Функция create_search_index принимает три параметра: data, chunk_size и chunk_overlap.
# Параметр data представляет собой текстовые данные, которые необходимо индексировать.
# Параметр chunk_size задает размер фрагмента текста, на который необходимо разбить данные, а chunk_overlap - насколько фрагменты будут перекрываться.
# В функции данные сначала разделяются на части по разделителю '\n\n', предполагая разделение на блоки диалогов или абзацы.
# Для каждой части определяется, кто является говорящим (Менеджер или Клиент), и этот тег добавляется к каждому фрагменту текста в последующих операциях.
# Функция использует объект splitter (экземпляр NLTKTextSplitter), чтобы разбить каждую часть текста на фрагменты согласно заданным параметрам chunk_size и chunk_overlap.
# Далее, к каждому фрагменту текста добавляется соответствующий тег говорящего (если это не первый фрагмент в части), и создается объект Document с этим фрагментом текста.
# Все объекты Document собираются в список source_chunks.
# Наконец, используя FAISS и класс OpenAIEmbeddings для векторизации, функция создает и возвращает индекс для быстрого поиска по текстовым фрагментам, хранящимся в source_chunks.
def create_search_index(data, chunk_size, chunk_overlap):
    source_chunks = []
    splitter = NLTKTextSplitter(chunk_size=chunk_size)
    parts = data.split('\n\n')
    last_speaker = ""

    for part in parts:
        speaker_tag = " Менеджер: " if " Менеджер: " in part else " Клиент: "
        last_speaker = part[:len(speaker_tag)]

        for i, chunk in enumerate(splitter.split_text(part)):
            content = last_speaker + chunk if i > 0 else chunk
            source_chunks.append(Document(page_content=content))
        embeddings_model = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

    return FAISS.from_documents(source_chunks, embeddings_model)

# Функция num_tokens_from_string определяет количество токенов в строке
#   string        - исходная строка
#   encoding_name - кодировка символов
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

# Функция num_tokens_from_messages определяет количество токенов в промпте
#   message - набор инструкций system, user и т.д.
#   model   - модель GPT
def num_tokens_from_messages(messages, model='gpt-4o-mini'):
    encoding = tiktoken.get_encoding('cl100k_base')
    token_counts = {
        MODEL: (4, -1),
        "default": (3, 1)
    }
    tokens_per_message, tokens_per_name = token_counts.get(model, token_counts["default"])
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message + sum(len(encoding.encode(value)) for key, value in message.items())
        if "name" in message:
            num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with assistant
    return num_tokens

# Функция answer_user_question загружает текст базы знаний из указанного URL,
# создает индекс поиска для этого текста и затем вызывает функцию answer_index,
# чтобы сгенерировать ответ на основе документа системы и заданной темы
#   system_doc_text     - инструкция system
#   knowledge_base_url  - путь к текстовому файлу базы знаний
#   topic               - ключевые фразы, по ним выбираем наиболее релевантные отрезки из базы знаний
#   instruction         - инструкция для формирования роли user
#   temperature         - температура
#   verbose             - показывать найденные чанки (в данном проекте не используется, оставлена для совместимости с кодом)
#   k                   - количество релевантных чанков
#   chunk_size          - максимальный размер чанка для формаирования индексной базы
#   chunk_overlap       - перекрытие чанков при формировании индексной базы
#   model               - модель GPT
#   example             - пример ответа (необязательный)
#   format              - формат примера ответа 'json' или 'text;
def answer_user_question(system_doc_text, knowledge_base_url, topic, instructions, temperature, verbose, k, chunk_size, chunk_overlap, model, example = '', format = 'text'):
    knowledge_base_text = load_document_text(knowledge_base_url)
    knowledge_base_index = create_search_index(knowledge_base_text, chunk_size, chunk_overlap)
    return answer_index(system_doc_text, topic, instructions, knowledge_base_index, temperature, verbose, k, model, example, format)

# Функция answer_index выполняет поиск по индексу для получения k
# наиболее релевантных документов по заданной теме, формирует
# сообщения для модели и генерирует ответ с использованием OpenAI API
#   system              - инструкция system
#   topic               - ключевые фразы, по ним выбираем наиболее релевантные отрезки из базы знаний
#   instruction         - инструкция для формирования роли user
#   search_index        - индексная база
#   temp                - температура
#   verbose             - показывать найденные чанки (в данном проекте не используется, оставлена для совместимости с кодом)
#   k                   - количество релевантных чанков
#   model               - модель GPT
#   example             - пример ответа
#   format              - формат примера ответа 'json' или 'text'
def answer_index(system, topic, instructions, search_index, temp, verbose, k, model, example = '', format = 'text'):
    docs = search_index.similarity_search_with_score(topic, k=k)
    response_format = None
    message_content = '\n '.join([f'Отрывок текста №{i+1}\n{doc[0].page_content}' for i, doc in enumerate(docs)])
    messages = [{"role": "system", "content": system}]
    if example != '':
      messages.append({"role": "user", "content": 'Ответь на вопрос' + ' и верни ответ в формате JSON' if format == 'json' else ''})
      messages.append({"role": "assistant", "content": example})
      if format == 'json': response_format = {'type': 'json_object'}
    messages.append({"role": "user", "content": f"{instructions}\n\nТексты для анализа:\n{message_content}"})
    client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
    completion = client.chat.completions.create(model = model, messages = messages, temperature = temp, response_format = response_format)
    return completion.choices[0].message.content

# Функция answer_user_question_from_answer формирует сообщения для модели
# на основе документа системы, инструкций и результатов анализа,
# затем генерирует ответ с использованием OpenAI API
#   system              - инструкция system
#   instruction         - инструкция для формирования роли user
#   answer_content      - результаты предыдущего анализа
#   temp                - температура
#   verbose             - показывать найденные чанки (в данном проекте не используется, оставлена для совместимости с кодом)
#   model               - модель GPT
def answer_user_question_from_answer(system, instructions, answers_content, temp, verbose, model):
    messages = [{"role": "system", "content": system}, {"role": "user", "content": f"{instructions}\n\nРезультаты анализа:\n{answers_content}"}]
    client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
    completion = client.chat.completions.create(model=model, messages=messages, temperature=temp)
    return completion.choices[0].message.content

# Функция переноса теста в output ячейках, для улучшения отображения вывода
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Функция merge_text_files объединяет содержимое двух текстовых файлов,
# добавляя строки из второго файла в конец первого файла
# с обновленной временной отметкой
#   file1_path - путь к первому файлу
#   file2_path - путь ко второму файлу
def merge_text_files(file1_path, file2_path):

    # открываем оба файла
    with open(file1_path, 'r+') as file1, open(file2_path, 'r') as file2:

        # Читаем содержимое первого файла
        file1_lines = file1.readlines()

        # Удаляем пустую строку или символ переноса строки в конце файла, если они есть
        if file1_lines[-1].strip() == '':
            file1_lines.pop()

        # Получаем временную отметку последней строки первого файла
        last_timestamp = file1_lines[-1].split()[0]

        # Читаем содержимое второго файла
        file2_lines = file2.readlines()

        # Обрабатываем каждую строку второго файла
        for line in file2_lines:
            line = line.strip()
            if line == '':
                continue  # Пропускаем пустые строки во втором файле

            timestamp, dialogue = line.split(' ', 1)
            minutes, seconds = map(int, timestamp.split(':'))

            # Если количество секунд больше 59, переходим к следующей минуте
            if seconds > 59:
                minutes += seconds // 60
                seconds %= 60

            # Переводим временную отметку в минуты и секунды
            total_minutes = minutes + int(last_timestamp[:2])
            total_seconds = seconds + int(last_timestamp[3:])

            # Проверяем, если количество секунд превышает 59, добавляем минуту и обновляем секунды
            if total_seconds > 59:
                total_minutes += total_seconds // 60
                total_seconds %= 60

            # Вычисляем время в формате HH:MM
            new_timestamp = f"{total_minutes:02d}:{total_seconds:02d}"

            # Записываем строку с обновленной временной отметкой в первый файл
            file1.write(f"\n\n{new_timestamp} {dialogue}")

        # Добавляем символ переноса строки в конце файла 1, если он отсутствует
        if not file1_lines[-1].endswith('\n'):
            file1.write('\n')


In [7]:
#@title Введите ссылку на папку Audio Record в папке zoom презентации с Яндекс диска c текстовыми файлами диалога
folder_url_yadisk = "https://disk.yandex.ru/d/IqOQeMnvd6YSKg" #@param {type:"string"} # Ссылка на основную папку на Яндекс Диске
optional_folder_url_yadisk2 = "" #@param {type:"string"} # Ссылка на дополнительную папку (опционально)
full_url = f'https://getfile.dokpub.com/yandex/get/{folder_url_yadisk}' # Построение полной ссылки для загрузки

# Проверка наличия дополнительной ссылки и построение полной ссылки для неё
if len(optional_folder_url_yadisk2):
  full_url_opt = f'https://getfile.dokpub.com/yandex/get/{optional_folder_url_yadisk2}'

temp_dir = 'temp' # Название временной директории

# Функция для загрузки содержимого по URL
def download_from_url(full_url, temp_dir, opt=False):
  # Создание временной директории, если она не существует
  if not os.path.exists(temp_dir):
    os.mkdir(temp_dir)

  # Запись URL во временный файл
  with open("temp/tmp.txt", "w") as f:
      f.write(full_url)

  try:
    # Удаление существующих файлов и папок, если они существуют
    try:
      os.remove("temp.zip")
      if not opt:
        !rm -R '/content/Audio Record/'
      !rm -R '/content/temp/Audio Record/'
    except:
      pass

    # Загрузка и распаковка архива
    !wget -O temp.zip -i temp/tmp.txt
    if opt:
      !unzip '/content/temp.zip' -d '/content/temp/'
    else:
      !unzip '/content/temp.zip' -d '/content/'
    output.clear()
    print('Файлы успешно загружены!')
  except Exception as e:
    print("Ошибка: ", e)

# Загрузка файлов из основной и дополнительной папок
download_from_url(full_url, temp_dir, opt=False)
if len(optional_folder_url_yadisk2):
  download_from_url(full_url_opt, temp_dir, opt=True)

# Пути к текстовым файлам в загруженных папках
base_text_path = '/content/Audio Record'
if len(optional_folder_url_yadisk2):
  base_opt_text_path = '/content/temp/Audio Record'
  # Слияние текстовых файлов с одинаковыми именами
  for first_file in os.listdir(base_text_path):
    file1_path = os.path.join(base_text_path, first_file)
    for second_file in os.listdir(base_opt_text_path):
      file2_path = os.path.join(base_opt_text_path, second_file)
      # Слияние файлов по типам (диалог, клиент, менеджер)
      if '_dialogue' in first_file and '_dialog_' in second_file:
        merge_text_files(file1_path, file2_path)
      elif 'client_' in first_file and '_client_' in second_file:
        merge_text_files(file1_path, file2_path)
      elif 'manager_' in first_file and '_manager_' in second_file:
        merge_text_files(file1_path, file2_path)
  print("Текстовые файлы двух записей объеденены!")

# Определение путей к текстовым файлам и их категоризация
text_path = [os.path.join(base_text_path, file) for file in os.listdir(base_text_path) if file.endswith('.txt')]
base = ''   # Диалог
base1 = '' # Клиент
base2 = '' # Менеджер

# Проверка количества текстовых файлов и их распределение по категориям
if len(text_path) == 3:
  for i_text in text_path:
    if '_dialogue' in i_text:
      base = i_text       # в итоге тут будет храниться путь к текстовому файлу
      print(i_text)
    elif 'client_' in i_text:
      base1 = i_text
      print(i_text)
    elif 'manager_' in i_text:
      base2 = i_text
      print(i_text)
else:
  print('Внимание!!! Количество текстовых файлов не соответствуют количеству равному 3 для правильной их обработки.\nПриведите их в правильный вид.')


Файлы успешно загружены!
/content/Audio Record/merged_dialogue.txt
/content/Audio Record/manager_recogn.txt
/content/Audio Record/client_recogn.txt


In [8]:
#@title Подсчет токенов и их соотношение в диалоге между клиентом и менджером
num_token_dialog = num_tokens_from_string(load_document_text(base), encoding_name="cl100k_base")
num_token_client = num_tokens_from_string(load_document_text(base1), encoding_name="cl100k_base")
num_token_manager = num_tokens_from_string(load_document_text(base2), encoding_name="cl100k_base")
print('Токенов в Диалог: ', num_token_dialog)
print('Токенов в Client +: ', num_token_client)
print('Токенов в Manager +: ', num_token_manager)
print("Соотношение токенов Клиент/Менеджер %: ", int((num_token_client*100)/num_token_dialog), '/', 100-int((num_token_client*100)/num_token_dialog))

Токенов в Диалог:  12107
Токенов в Client +:  3385
Токенов в Manager +:  8701
Соотношение токенов Клиент/Менеджер %:  27 / 73


In [27]:
#@title 1\. Генерация ответа с поиском по контексту для "Тарифы"
model = "deepseek/deepseek-r1-distill-qwen-32b:free" #@param ["deepseek/deepseek-r1-distill-qwen-32b:free", "gpt-4", "gpt-3.5-turbo-0125"]
content = 'Manager +' #@param ['Диалог', 'Client +', 'Manager +']
chunk_size = 536 #@param {type: "slider", min: 0, max: 1024, step:8}
chunk_overlap = 0 #@param {type: "slider", min: 0, max: 256, step:8}
temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}
num_fragment = 12 #@param {type:"integer"}
system_prompt = "\u0422\u044B \u043E\u0442\u043B\u0438\u0447\u043D\u043E \u0443\u043C\u0435\u0435\u0448\u044C \u0430\u043D\u0430\u043B\u0438\u0437\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u0442\u0435\u043A\u0441\u0442 \u0438 \u043D\u0430\u0445\u043E\u0434\u0438\u0442\u044C \u0432 \u0442\u0435\u043A\u0441\u0442\u0430\u0445 \u0442\u0430\u0440\u0438\u0444\u044B. \u0422\u044B \u0437\u043D\u0430\u0435\u0448\u044C \u0447\u0442\u043E \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u044F \u043C\u043E\u0436\u0435\u0442 \u043F\u0440\u0435\u0434\u043E\u0441\u0442\u0430\u0432\u0438\u0442\u044C \u0442\u0430\u043A\u0438\u0435 \u0442\u0430\u0440\u0438\u0444\u044B \u043D\u043E \u043F\u0440\u0438 \u044D\u0442\u043E\u043C \u0438\u0445 \u043C\u043E\u0436\u0435\u0442 \u043D\u0435 \u0431\u044B\u0442\u044C \u0432 \u0442\u0435\u043A\u0441\u0442\u0430\u0445:  \"\u0431\u0430\u0437\u043E\u0432\u044B\u0439 \u0442\u0430\u0440\u0438\u0444\", \"\u043F\u0440\u043E\u0434\u0432\u0438\u043D\u0443\u0442\u044B\u0439 \u0442\u0430\u0440\u0438\u0444\", \"\u043E\u0441\u043D\u043E\u0432\u043D\u043E\u0439 \u0442\u0430\u0440\u0438\u0444\", \"\u0440\u0430\u0441\u0448\u0438\u0440\u0435\u043D\u043D\u044B\u0439 \u0442\u0430\u0440\u0438\u0444\", \"\u043F\u0430\u0439\u0442\u043E\u043D-\u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\", \"\u044D\u043A\u0441\u043F\u0440\u0435\u0441\u0441-\u043A\u0443\u0440\u0441 \u043F\u043E \u0410\u0419/\u044D\u0439\u0430\u0439\", \"\u0441\u0442\u0430\u0436\u0438\u0440\u043E\u0432\u043A\u0438\", \"\u043D\u043E\u0432\u044B\u0439 \u0442\u0430\u0440\u0438\u0444\", \"\u0444\u0440\u0435\u0439\u043C\u0432\u043E\u0440\u043A\u0438\", \"\u0447\u0430\u0442  \u0434\u0436\u0438\u043F\u0438\u0442\u0438 (\u0434\u0436\u0438\u043F\u0438\u0442\u0438 = \u0447\u0430\u0441 \u043F\u044F\u0442\u0438/\u0447\u0430\u0442 \u043F\u044F\u0442\u0438/\u0447\u0430\u0442 \u0438 \u043F\u044F\u0442\u044C/\u0447\u0430\u0442 \u0434\u0435\u0442\u0438)\" . \u0422\u044B \u0443\u043A\u0430\u0437\u044B\u0432\u0430\u0435\u0448\u044C \u0442\u0430\u0440\u0438\u0444\u044B \u0442\u043E\u043B\u044C\u043A\u043E \u0435\u0441\u043B\u0438 \u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0442\u0435\u043B\u044C\u043D\u043E \u043D\u0430\u0448\u0435\u043B \u0438\u0445 \u0432 \u0442\u0435\u043A\u0441\u0442\u0430\u0445. " #@param {type:"string"}
instructions = "\u041F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430 \u0434\u0430\u0432\u0430\u0439 \u043F\u043E\u0434\u0443\u043C\u0430\u0435\u043C \u0448\u0430\u0433 \u0437\u0430 \u0448\u0430\u0433\u043E\u043C. \u041D\u0430\u0439\u0434\u0438 \u0432 \u0442\u0435\u043A\u0441\u0442\u0430\u0445 \u0443\u043F\u043E\u043C\u0438\u043D\u0430\u043D\u0438\u0435 \u0442\u0430\u0440\u0438\u0444\u043E\u0432 \u0438 \u0441\u0434\u0435\u043B\u0430\u0439 \u043E\u0431 \u044D\u0442\u043E\u043C \u043E\u0442\u0447\u0435\u0442 \u0441\u043B\u0435\u0434\u0443\u044E\u0449\u0435\u0433\u043E \u0441\u043E\u0434\u0435\u0440\u0436\u0430\u043D\u0438\u044F: \u041D\u0430\u043F\u0438\u0448\u0438 \u0441\u043F\u0438\u0441\u043A\u043E\u043C \u0442\u0435 \u0442\u0430\u0440\u0438\u0444\u044B, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0442\u043E\u0447\u043D\u043E \u0443\u043F\u043E\u043C\u0438\u043D\u0430\u044E\u0442\u0441\u044F \u0432 \u0442\u0435\u043A\u0441\u0442\u0430\u0445: \"\u041D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u0442\u0430\u0440\u0438\u0444\u0430\"+\u0432\u0440\u0435\u043C\u044F, \u043A\u043E\u0433\u0434\u0430 \u0431\u044B\u043B \u043D\u0430\u0437\u0432\u0430\u043D \u0442\u0430\u0440\u0438\u0444 (23:19)+\u043A\u043E\u0440\u043E\u0442\u043A\u0430\u044F \u0446\u0438\u0442\u0430\u0442\u0430 \u0432\u044B\u0441\u043A\u0430\u0437\u044B\u0432\u0430\u043D\u0438\u044F \u043E \u0442\u0430\u0440\u0438\u0444\u0435,   \u041F\u0440\u0438\u043C\u0435\u0447\u0430\u043D\u0438\u0435:  \u0435\u0441\u043B\u0438 \u0442\u0430\u0440\u0438\u0444 \u043D\u0430\u0437\u044B\u0432\u0430\u043B\u0441\u044F \u043D\u0435\u0441\u043A\u043E\u043B\u044C\u043A\u043E \u0440\u0430\u0437, \u0442\u043E \u043D\u0430\u043F\u0438\u0448\u0438 \u0435\u0433\u043E \u0442\u043E\u043B\u044C\u043A\u043E \u043E\u0434\u0438\u043D \u0440\u0430\u0437 \u0432 \u043F\u0435\u0440\u0432\u043E\u043C \u043E\u0442\u0447\u0435\u0442\u0435.  " #@param {type:"string"}
topicphrase = '\u0442\u0430\u0440\u0438\u0444, \u043E\u0441\u043D\u043E\u0432\u043D\u043E\u0439, \u0431\u0430\u0437\u043E\u0432\u044B\u0439, \u0440\u0430\u0441\u0448\u0438\u0440\u0435\u043D\u043D\u044B\u0439, \u043F\u0440\u043E\u0435\u043A\u0442 \u043F\u043E\u0434 \u043A\u043B\u044E\u0447, \u043A\u0443\u0440\u0441, \u043F\u0440\u043E\u0434\u0432\u0438\u043D\u0443\u0442\u044B\u0439, \u043F\u0440\u043E\u0435\u043A\u0442 \u043F\u043E\u0434 \u043A\u043B\u044E\u0447, \u043F\u0430\u0439\u0442\u043E\u043D \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A, \u0441\u0442\u0430\u0436\u0438\u0440\u043E\u0432\u043A\u0430, \u0444\u0440\u0435\u0439\u043C\u0432\u043E\u0440\u043A, \u0421\u043E\u0437\u0434\u0430\u043D\u0438\u0435 \u043D\u0435\u0439\u0440\u043E\u043D\u043D\u044B\u0445 \u0441\u0435\u0442\u0435\u0439 \u0441 \u043F\u043E\u043C\u043E\u0449\u044C\u044E \u0447\u0430\u0442\u0430  GTP' #@param {type:"string"}

if content == 'Диалог': content_base = base
elif content == 'Client +': content_base = base1
elif content == 'Manager +': content_base = base2

out = {}
out['тарифы_1'] = answer_user_question(
  system_prompt, content_base, topicphrase,
  instructions, temperature, 1, num_fragment,
  chunk_size, chunk_overlap, model
)

pprint(f'system_prompt: {system_prompt}')
print()
pprint(f'instructions: {instructions}')
print()
pprint(f'topicphrase: {topicphrase}')
print("\nОтвет:\n", out['тарифы_1'])

('system_prompt: Ты отлично умеешь анализировать текст и находить в текстах '
 'тарифы. Ты знаешь что компания может предоставить такие тарифы но при этом '
 'их может не быть в текстах:  "базовый тариф", "продвинутый тариф", "основной '
 'тариф", "расширенный тариф", "пайтон-разработчик", "экспресс-курс по '
 'АЙ/эйай", "стажировки", "новый тариф", "фреймворки", "чат  джипити (джипити '
 '= час пяти/чат пяти/чат и пять/чат дети)" . Ты указываешь тарифы только если '
 'действительно нашел их в текстах. ')

('instructions: Пожалуйста давай подумаем шаг за шагом. Найди в текстах '
 'упоминание тарифов и сделай об этом отчет следующего содержания: Напиши '
 'списком те тарифы, которые точно упоминаются в текстах: "Название '
 'тарифа"+время, когда был назван тариф (23:19)+короткая цитата высказывания о '
 'тарифе,   Примечание:  если тариф назывался несколько раз, то напиши его '
 'только один раз в первом отчете.  ')

('topicphrase: тариф, основной, базовый, расширенный, проект под ключ,

In [38]:
#@title 2\. Генерация ответа с поиском по контексту для "ПУШ"
model = "deepseek/deepseek-r1-distill-qwen-32b:free" #@param ["deepseek/deepseek-r1-distill-qwen-32b:free", "gpt-4", "gpt-3.5-turbo-0125"]
content = 'Manager +' #@param ['Диалог', 'Client +', 'Manager +']
chunk_size = 760 #@param {type: "slider", min: 0, max: 1024, step:8}
chunk_overlap = 0 #@param {type: "slider", min: 0, max: 256, step:8}
temperature = 0.1 #@param {type: "slider", min: 0, max: 1, step:0.1}
num_fragment = 5 #@param {type:"integer"}
system_prompt = "\u0422\u044B \u0441\u0430\u043C\u044B\u0439 \u043B\u0443\u0447\u0448\u0438\u0439 \u0441\u043E\u0442\u0440\u0443\u0434\u043D\u0438\u043A \u043E\u0442\u0434\u0435\u043B\u0430 \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u044F \u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0430 \u043E\u0431\u0449\u0435\u043D\u0438\u044F \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440\u0430 \u043F\u043E \u043F\u0440\u043E\u0434\u0430\u0436\u0430\u043C \u0438 \u043A\u043B\u0438\u0435\u043D\u0442\u0430. \u041C\u0435\u043D\u0435\u0434\u0436\u0435\u0440 \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u0432 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u043F\u0440\u043E\u0434\u0430\u0435\u0442 \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u0435 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044E \u043D\u0430 python \u0438 \u043D\u0435\u0439\u0440\u043E\u043D\u043D\u044B\u043C \u0441\u0435\u0442\u044F\u043C. \u0422\u044B \u0432\u0441\u0435\u0433\u0434\u0430 \u043E\u0447\u0435\u043D\u044C \u0442\u043E\u0447\u043D\u043E \u0441\u043B\u0435\u0434\u0443\u0435\u0448\u044C \u043F\u043E\u0440\u044F\u0434\u043A\u0443 \u043E\u0442\u0447\u0435\u0442\u0430.  \u0422\u044B \u0437\u043D\u0430\u0435\u0448\u044C \u0447\u0442\u043E \"\u043F\u0443\u0448\" - \u044D\u0442\u043E \u043F\u0440\u044F\u043C\u043E\u0439 \u0438 \u044F\u0432\u043D\u044B\u0439 \u043F\u0440\u0438\u0437\u044B\u0432 \u043A \u043E\u043F\u043B\u0430\u0442\u0435/ \u043F\u0440\u0438\u0437\u044B\u0432 \u043A \u0432\u044B\u0431\u043E\u0440\u0443 \u0442\u0430\u0440\u0438\u0444\u0430 / \u043F\u0440\u0438\u0437\u044B\u0432 \u043A \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u044E.  \u0422\u0432\u043E\u044F \u043F\u0440\u044F\u043C\u0430\u044F \u0437\u0430\u0434\u0430\u0447\u0430 - \u043D\u0430\u0445\u043E\u0434\u0438\u0442\u044C \u044F\u0432\u043D\u044B\u0435 \u043F\u0443\u0448 \u0432 \u0442\u0435\u043A\u0441\u0442\u0430\u0445 \u0438 \u043F\u0438\u0441\u0430\u0442\u044C \u043F\u043E \u043D\u0438\u043C \u0437\u0430\u0434\u0430\u043D\u043D\u044B\u0435 \u043E\u0442\u0447\u0435\u0442\u044B." #@param {type:"string"}
instructions = "Пожалуйста давайте подумаем шаг за шагом. Сделай 3 отчета по наличию пуш в текстах.  Напишите только то что описано в порядке отчетов. Отчет №01: Найди в текстах все очень явные пуш. Напиши один список самых явных пуш пунктами: \"Цитаты пуш + время отрывка, по которому ты делаешь вывод (например, 23:19 отдельным элементом словаря)\". Не забудь указать тайминг, когда менеджером произнесен пуш. Отчет №02: общая оценка качества описания от 0% до 100% (100% - когда более трех пуш найдены в текстах и  пуш отличный способный убедить присоединиться к курсам и произвести оплату, 50% - описание пуш было, но не убедительное, 0% - описания пуш не было). Отчет №03:  посчитай количество пуш.  Примечание: тебе запрещено писать в первом отчете пуш, которые повторяются." #@param {type:"string"}
topicphrase = '\u0432\u044B\u0431\u0438\u0440\u0430\u0435\u0442\u0435, \u0433\u043E\u0442\u043E\u0432\u044B, \u041E\u043F\u043B\u0430\u0442\u0430, \u0440\u0430\u0441\u0441\u0440\u043E\u0447\u043A\u0430, \u043A\u0440\u0435\u0434\u0438\u0442, \u0441\u0442\u043E\u0438\u043C\u043E\u0441\u0442\u044C, \u0446\u0435\u043D\u0430, \u0442\u0430\u0440\u0438\u0444, \u043F\u043B\u0430\u0442\u0435\u0436, \u0441\u043A\u0438\u0434\u043A\u0430, \u043F\u0440\u0438\u0441\u043E\u0435\u0434\u0438\u043D\u0438\u0442\u0435\u0441\u044C, \u043F\u0440\u0438\u0433\u043B\u0430\u0448\u0430\u044E, \u0447\u0442\u043E \u0441\u043A\u0430\u0436\u0435\u0442\u0435?, \u041A\u0430\u043A\u043E\u0439 \u0442\u0430\u0440\u0438\u0444 \u043D\u0440\u0430\u0432\u0438\u0442\u0441\u044F?' #@param {type:"string"}
answer_example = "'reports': [{'num_report':  int, 'name_report': str, 'text_report': str},...]" #@param {type:"string"}
answer_format = "json" #@param ["json", "text"]

if content == 'Диалог': content_base = base
elif content == 'Client +': content_base = base1
elif content == 'Manager +': content_base = base2

result = answer_user_question(
  system_prompt, content_base, topicphrase,
  instructions, temperature, 1, num_fragment,
  chunk_size, chunk_overlap, model, answer_example, answer_format
)
out['тарифы_2'] = json.loads(result.replace("```json", "").replace("```", ""))
out['тарифы_2_total'] = ' '.join(item['quote'] for item in out['тарифы_2']['reports'][0]['text_report'])

pprint(f'system_prompt: {system_prompt}')
print()
pprint(f'instructions: {instructions}')
print()
pprint(f'topicphrase: {topicphrase}')
print('\nОтвет:\n', result)
print('\nДля итогового отчета:\n', out['тарифы_2_total'])

('system_prompt: Ты самый лучший сотрудник отдела контроля качества общения '
 'менеджера по продажам и клиента. Менеджер работает в компании, которая '
 'продает обучение программированию на python и нейронным сетям. Ты всегда '
 'очень точно следуешь порядку отчета.  Ты знаешь что "пуш" - это прямой и '
 'явный призыв к оплате/ призыв к выбору тарифа / призыв к обучению.  Твоя '
 'прямая задача - находить явные пуш в текстах и писать по ним заданные '
 'отчеты.')

('instructions: Пожалуйста давайте подумаем шаг за шагом. Сделай 3 отчета по '
 'наличию пуш в текстах.  Напишите только то что описано в порядке отчетов. '
 'Отчет №01: Найди в текстах все очень явные пуш. Напиши один список самых '
 'явных пуш пунктами: "Цитаты пуш + время отрывка, по которому ты делаешь '
 'вывод (например, 23:19 отдельным элементом словаря)". Не забудь указать '
 'тайминг, когда менеджером произнесен пуш. Отчет №02: общая оценка качества '
 'описания от 0% до 100% (100% - когда более трех пуш найдены в 

In [46]:
#@title 3\. Варианты оплат
model = "deepseek/deepseek-r1-distill-qwen-32b:free" #@param ["deepseek/deepseek-r1-distill-qwen-32b:free", "gpt-4", "gpt-3.5-turbo-0125"]
content = 'Диалог' #@param ['Диалог', 'Client +', 'Manager +']
chunk_size = 616 #@param {type: "slider", min: 0, max: 1024, step:8}
chunk_overlap = 0 #@param {type: "slider", min: 0, max: 256, step:8}
temperature = 0.1 #@param {type: "slider", min: 0, max: 1, step:0.1}
num_fragment = 8 #@param {type:"integer"}
system_prompt = "\u0422\u044B \u0441\u0430\u043C\u044B\u0439 \u043B\u0443\u0447\u0448\u0438\u0439 \u0441\u043E\u0442\u0440\u0443\u0434\u043D\u0438\u043A \u043E\u0442\u0434\u0435\u043B\u0430 \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u044F \u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0430 \u043E\u0431\u0449\u0435\u043D\u0438\u044F \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440\u0430 \u043F\u043E \u043F\u0440\u043E\u0434\u0430\u0436\u0430\u043C \u0438 \u043A\u043B\u0438\u0435\u043D\u0442\u0430. \u041C\u0435\u043D\u0435\u0434\u0436\u0435\u0440 \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u0432 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u043F\u0440\u043E\u0434\u0430\u0435\u0442 \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u0435 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044E \u043D\u0430 python \u0438 \u043D\u0435\u0439\u0440\u043E\u043D\u043D\u044B\u043C \u0441\u0435\u0442\u044F\u043C. " #@param {type:"string"}
instructions = "Какие варианты оплаты обучения или тарифа предлагал менеджер(полная оплата, рассрочка или кредит)? Порядок отчета: \"вариант оплаты, который найден в текстах+время когда была назван вариант оплаты\". Дублирующиеся варианты оплат - не включай в отчет. Не упоминай в ответе зарплаты разработчиков. " #@param {type:"string"}
topicphrase = '\u041E\u043F\u043B\u0430\u0442\u0430. \u0440\u0430\u0441\u0441\u0440\u043E\u0447\u043A\u0430, \u043A\u0440\u0435\u0434\u0438\u0442, \u0441\u0442\u043E\u0438\u043C\u043E\u0441\u0442\u044C, \u0446\u0435\u043D\u0430, \u043F\u043B\u0430\u0442\u0435\u0436, \u0441\u043A\u0438\u0434\u043A\u0430, \u043F\u0440\u0438\u0441\u043E\u0435\u0434\u0438\u043D\u0438\u0442\u0435\u0441\u044C, \u043F\u0440\u0438\u0433\u043B\u0430\u0448\u0430\u044E, \u0447\u0442\u043E \u0441\u043A\u0430\u0436\u0435\u0442\u0435? \u041A\u0430\u043A\u043E\u0439 \u0442\u0430\u0440\u0438\u0444 \u043D\u0440\u0430\u0432\u0438\u0442\u0441\u044F?' #@param {type:"string"}

if content == 'Диалог': content_base = base
elif content == 'Client +': content_base = base1
elif content == 'Manager +': content_base = base2

out['тарифы_3'] = answer_user_question(
  system_prompt, content_base, topicphrase,
  instructions, temperature, 1, num_fragment,
  chunk_size, chunk_overlap, model
)

pprint(f'system_prompt: {system_prompt}')
print()
pprint(f'instructions: {instructions}')
print()
pprint(f'topicphrase: {topicphrase}')
print("\nОтвет:\n", out['тарифы_3'])

('system_prompt: Ты самый лучший сотрудник отдела контроля качества общения '
 'менеджера по продажам и клиента. Менеджер работает в компании, которая '
 'продает обучение программированию на python и нейронным сетям. ')

('instructions: Какие варианты оплаты обучения или тарифа предлагал '
 'менеджер(полная оплата, рассрочка или кредит)? Порядок отчета: "вариант '
 'оплаты, который найден в текстах+время когда была назван вариант оплаты". '
 'Дублирующиеся варианты оплат - не включай в отчет. Не упоминай в ответе '
 'зарплаты разработчиков. ')

('topicphrase: Оплата. рассрочка, кредит, стоимость, цена, платеж, скидка, '
 'присоединитесь, приглашаю, что скажете? Какой тариф нравится?')

Ответ:
 В предоставленных текстах варианты оплаты обучения или тарифы не упоминаются.


На основе записи диалога Клиента и Менеджера, используя материалы занятия, проанализируйте предложенные тарифы, пуш-уведомления и варианты оплаты, а затем составьте итоговый отчет на основе проведенного анализа. Ссылка на запись диалога Менеджера и Клиента: https://disk.yandex.ru/d/IqOQeMnvd6YSKg

In [47]:
#@title Отчет об эффективности менеджера в выявлении потребностей клиента

# параметры запроса
model = "deepseek/deepseek-r1-distill-qwen-32b:free" #@param ["deepseek/deepseek-r1-distill-qwen-32b:free", "gpt-4", "gpt-3.5-turbo-0125"]
temperature = 0.1 #@param {type: "slider", min: 0, max: 1, step:0.1}
system_prompt = "\u0422\u044B \u043E\u0442\u043B\u0438\u0447\u043D\u043E \u0440\u0430\u0437\u0431\u0438\u0440\u0430\u0435\u0448\u044C\u0441\u044F \u0432 \u0430\u043D\u0430\u043B\u0438\u0437\u0435 \u0438 \u0434\u0435\u043B\u0430\u0435\u0448\u044C \u0432\u044B\u0432\u043E\u0434\u044B. \u0422\u044B \u043F\u043E\u043D\u0438\u043C\u0430\u0435\u0448\u044C, \u043A\u0430\u043A \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0438\u0442\u044C \u043F\u043E\u0442\u0440\u0435\u0431\u043D\u043E\u0441\u0442\u0438. \u0422\u0432\u043E\u0438 \u043E\u0442\u0447\u0435\u0442\u044B \u0432\u0441\u0435\u0433\u0434\u0430 \u043A\u0440\u0430\u0442\u043A\u0438 \u0438 \u043F\u043E \u0441\u0443\u0449\u0435\u0441\u0442\u0432\u0443. \u0422\u044B \u043E\u0442\u043B\u0438\u0447\u043D\u043E \u043F\u043E\u0434\u0447\u0435\u0440\u043A\u0438\u0432\u0430\u0435\u0448\u044C \u0432\u0430\u0436\u043D\u044B\u0435 \u043C\u043E\u043C\u0435\u043D\u0442\u044B \u0432 \u0442\u0435\u043A\u0441\u0442\u0430\u0445 \u0434\u043B\u044F \u043E\u0442\u0447\u0435\u0442\u0430 \u043E\u0442\u0434\u0435\u043B\u0430 \u043F\u0440\u043E\u0434\u0430\u0436. \u0422\u044B \u0432\u0441\u0435\u0433\u0434\u0430 \u043F\u0438\u0448\u0435\u0448\u044C \u043E\u0442\u0447\u0435\u0442 \u043E\u0431\u044A\u0435\u043C\u043E\u043C \u0434\u043E 1000 \u0441\u043B\u043E\u0432. \u0422\u044B \u0432\u0441\u0435\u0433\u0434\u0430 \u043E\u0447\u0435\u043D\u044C \u0442\u043E\u0447\u043D\u043E \u0441\u043B\u0435\u0434\u0443\u0435\u0448\u044C \u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u0435 \u043E\u0442\u0447\u0435\u0442\u0430." #@param {type:"string"}
instructions = "\u041F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430 \u0434\u0430\u0432\u0430\u0439\u0442\u0435 \u043F\u043E\u0434\u0443\u043C\u0430\u0435\u043C \u0448\u0430\u0433 \u0437\u0430 \u0448\u0430\u0433\u043E\u043C. \u041D\u0430\u043F\u0438\u0448\u0438, \u043F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430, 4 \u043E\u0442\u0447\u0435\u0442\u0430 \u043F\u043E \u0432\u044B\u044F\u0432\u043B\u0435\u043D\u0438\u044E \u043F\u043E\u0442\u0440\u0435\u0431\u043D\u043E\u0441\u0442\u0435\u0439 \u043A\u043B\u0438\u0435\u043D\u0442\u0430, \u0441\u0442\u0440\u043E\u0433\u043E \u043A\u0430\u043A \u0443\u043A\u0430\u0437\u0430\u043D\u043E \u0432 \u044D\u0442\u0438\u0445 \u043F\u043E\u0440\u044F\u0434\u043A\u0430\u0445 \u043E\u0442\u0447\u0435\u0442\u043E\u0432: \"\u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442 \u0430\u043D\u0430\u043B\u0438\u0437\u0430 \u043F\u043E \u0432\u044B\u044F\u0432\u043B\u0435\u043D\u0438\u044E \u043F\u043E\u0442\u0440\u0435\u0431\u043D\u043E\u0441\u0442\u0435\u0439 \u043A\u043B\u0438\u0435\u043D\u0442\u0430\":    1 \u043E\u0442\u0447\u0435\u0442: \u041F\u043E\u0440\u044F\u0434\u043E\u043A \u043F\u0435\u0440\u0432\u043E\u0433\u043E \u043E\u0442\u0447\u0435\u0442\u0430:  \u043F\u0440\u043E\u0430\u043D\u0430\u043B\u0438\u0437\u0438\u0440\u0443\u0439 \u0441\u0440\u0430\u0437\u0443 \u0432\u0441\u0435 \u043F\u043E\u043B\u0443\u0447\u0435\u043D\u043D\u044B\u0435 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u044B \u0438 \u043D\u0430\u043F\u0438\u0448\u0438 \u043A\u0440\u0430\u0442\u043A\u043E \u043E\u0431\u0449\u0438\u0439 \u0432\u044B\u0432\u043E\u0434, \u043A\u0430\u043A \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440 \u0432\u044B\u044F\u0432\u043B\u044F\u043B \u043F\u043E\u0442\u0440\u0435\u0431\u043D\u043E\u0441\u0442\u0438 \u0438 \u043D\u0430\u0441\u043A\u043E\u043B\u044C\u043A\u043E \u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u043E \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440 \u0437\u0430\u043A\u0440\u044B\u043B \u043F\u043E\u0442\u0440\u0435\u0431\u043D\u043E\u0441\u0442\u0438, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0432\u044B\u0441\u043A\u0430\u0437\u0430\u043B \u043A\u043B\u0438\u0435\u043D\u0442, \u0430 \u0442\u0430\u043A\u0436\u0435  \u0432\u044B\u0434\u0435\u043B\u0438 \u043A\u043B\u044E\u0447\u0435\u0432\u044B\u0435 \u043C\u043E\u043C\u0435\u043D\u0442\u044B \u0438\u0437 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0430 \u0430\u043D\u0430\u043B\u0438\u0437\u0430.  2 \u043E\u0442\u0447\u0435\u0442:  \u041F\u043E\u0440\u044F\u0434\u043E\u043A \u0432\u0442\u043E\u0440\u043E\u0433\u043E \u043E\u0442\u0447\u0435\u0442\u0430: \u0441\u0443\u0434\u044F \u043F\u043E \u0430\u043D\u0430\u043B\u0438\u0437\u0443 \u043F\u0435\u0440\u0432\u043E\u0433\u043E \u043E\u0442\u0447\u0435\u0442\u0430 \u043D\u0430\u043F\u0438\u0448\u0438 \u043A\u0440\u0430\u0442\u043A\u0438\u0439 \u0441\u043F\u0438\u0441\u043E\u043A \u043A\u043E\u043D\u043A\u0440\u0435\u0442\u043D\u044B\u0445 \u0440\u0435\u043A\u043E\u043C\u0435\u043D\u0434\u0430\u0446\u0438\u0439 \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440\u0443 \u0434\u043B\u044F \u0443\u043B\u0443\u0447\u0448\u0435\u043D\u0438\u044F \u043D\u0430\u0432\u044B\u043A\u043E\u0432 \u0432\u044B\u044F\u0432\u043B\u0435\u043D\u0438\u044F \u043F\u043E\u0442\u0440\u0435\u0431\u043D\u043E\u0441\u0442\u0435\u0439.   3 \u043E\u0442\u0447\u0435\u0442: \u0421\u0434\u0435\u043B\u0430\u0439 \u0440\u0430\u0437\u0431\u043E\u0440 \u043F\u043E \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u0438 \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440\u0430  \u0432 \u0432\u044B\u044F\u0432\u043B\u0435\u043D\u0438\u0438 \u043F\u043E\u0442\u0440\u0435\u0431\u043D\u043E\u0441\u0442\u0435\u0439 \u043A\u043B\u0438\u0435\u043D\u0442\u0430, \u0441\u0443\u0434\u044F \u043F\u043E \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0430\u043C \u0430\u043D\u0430\u043B\u0438\u0437\u0430, \u0438 \u043D\u0430\u043F\u0438\u0448\u0438 \u044D\u0442\u0438 \u0432\u044B\u0432\u043E\u0434\u044B.  4 \u043E\u0442\u0447\u0435\u0442: \u041F\u043E\u0440\u044F\u0434\u043E\u043A \u0447\u0435\u0442\u0432\u0435\u0440\u0442\u043E\u0433\u043E \u043E\u0442\u0447\u0435\u0442\u0430: \u043D\u0430\u043F\u0438\u0448\u0438 \u0432 \u0442\u043E\u0447\u043D\u043E\u043C \u043F\u043E\u0440\u044F\u0434\u043A\u0435: \"quality\":  \u043F\u0443\u0442\u0435\u043C \u0432\u044B\u0447\u0438\u0441\u043B\u0435\u043D\u0438\u0439 \u043D\u0430\u043F\u0438\u0448\u0438 \u043E\u0434\u043D\u0443  \u0446\u0438\u0444\u0440\u0443 \u0441\u0440\u0435\u0434\u043D\u0435\u0433\u043E \u043F\u0440\u043E\u0446\u0435\u043D\u0442\u0430 \u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0430 \u043F\u043E \u0432\u0441\u0435\u043C \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0430\u043C \u0430\u043D\u0430\u043B\u0438\u0437\u0430 \u043E\u0442 0% \u0434\u043E 100%. \u041F\u0440\u0438\u043C\u0435\u0447\u0430\u043D\u0438\u0435: \u043E\u0431\u044F\u0437\u0430\u0442\u0435\u043B\u044C\u043D\u043E \u043D\u0430\u043F\u0438\u0448\u0438 \u0442\u043E\u043B\u044C\u043A\u043E 4 \u043E\u0442\u0447\u0435\u0442\u0430. " #@param {type:"string"}

# данные для запроса
data = {
    'выявление_1_1': 'Вопросы, заданные менеджером: Пример вопросов от менеджера.',
    'выявление_2_1': 'Список найденных потребностей: Найденные потребности клиента.',
    'закрытие_1_total': 'Что в диалоге говорилось о трудоустройстве: Обсуждение трудоустройства.',
    'закрытие_2_total': 'Что менеджер говорил о заработной плате: Информация о заработной плате.',
    'закрытие_3_total': 'Удаленка. Что менеджер говорил про удаленный формат работы: Комментарий про удаленный формат работы.',
    'закрытие_4_total': 'Что менеджер говорил про личное развитие: Комментарий про личное развитие.',
    'закрытие_5_total': 'Что менеджер говорил про развитие в IT сфере: Развитие в IT сфере.',
    'закрытие_6_total': 'Что менеджер говорил о смене деятельности: Обсуждение смены деятельности.',
    'закрытие_7_total': 'Где менеджер рассказывал про занятия нейронками как хобби: Информация про занятия нейронками как хобби.',
    'закрытие_8_total': 'Что менеджер говорил о карьерном росте: Комментарий о карьерном росте.',
    'закрытие_9_total': 'Что менеджер говорил про создание собственного проекта: Информация про создание собственного проекта.'
}
answers = [
    f"Анализ №{i + 1}. {data[key]} {out.get(key, '').strip()}"
    for i, key in enumerate(data.keys())
]
answers_text = " ".join(answers)

# выполнение запроса
out['выявление_отчет'] = answer_user_question_from_answer(
  system_prompt, instructions, answers, temperature, 1, model
)

# вывод результатов
pprint(f'system_prompt: {system_prompt}')
print()
pprint(f'instructions: {instructions}')
for i, answer in enumerate(answers):
  print()
  print(f'answer_{i + 1}: {answer[:300]}...')
print('\nОтвет:\n', out['выявление_отчет'])

('system_prompt: Ты отлично разбираешься в анализе и делаешь выводы. Ты '
 'понимаешь, как эффективно определить потребности. Твои отчеты всегда кратки '
 'и по существу. Ты отлично подчеркиваешь важные моменты в текстах для отчета '
 'отдела продаж. Ты всегда пишешь отчет объемом до 1000 слов. Ты всегда очень '
 'точно следуешь структуре отчета.')

('instructions: Пожалуйста давайте подумаем шаг за шагом. Напиши, пожалуйста, '
 '4 отчета по выявлению потребностей клиента, строго как указано в этих '
 'порядках отчетов: "результат анализа по выявлению потребностей клиента":    '
 '1 отчет: Порядок первого отчета:  проанализируй сразу все полученные '
 'результаты и напиши кратко общий вывод, как менеджер выявлял потребности и '
 'насколько качественно менеджер закрыл потребности, которые высказал клиент, '
 'а также  выдели ключевые моменты из результата анализа.  2 отчет:  Порядок '
 'второго отчета: судя по анализу первого отчета напиши краткий список '
 'конкретных рекомендаций мене

In [50]:
#@title Отчет итого
model = "deepseek/deepseek-r1-distill-qwen-32b:free" #@param ["deepseek/deepseek-r1-distill-qwen-32b:free", "gpt-4", "gpt-3.5-turbo-0125"]
temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}
system_prompt = "\u0422\u044B \u0432\u0435\u043B\u0438\u043A\u043E\u043B\u0435\u043F\u043D\u043E \u0443\u043C\u0435\u0435\u0448\u044C \u0430\u043D\u0430\u043B\u0438\u0437\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u0438 \u0434\u0435\u043B\u0430\u0442\u044C \u0432\u044B\u0432\u043E\u0434\u044B. \u0420\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u044B \u0430\u043D\u0430\u043B\u0438\u0437\u0430 - \u043F\u0440\u0435\u0434\u044B\u0434\u0443\u0449\u0438\u0435 \u0432\u044B\u0432\u043E\u0434\u044B \u043F\u043E \u043E\u0431\u0449\u0435\u043D\u0438\u044E \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440\u0430 \u0438 \u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0443 \u0435\u0433\u043E \u0440\u0430\u0431\u043E\u0442\u044B. \u0422\u044B \u043E\u0442\u043B\u0438\u0447\u043D\u043E \u0443\u043C\u0435\u0435\u0448\u044C \u0432\u044B\u0434\u0435\u043B\u044F\u0442\u044C \u0437\u043D\u0430\u0447\u0438\u043C\u044B\u0435 \u0432\u0435\u0449\u0438 \u0432 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0435 \u0430\u043D\u0430\u043B\u0438\u0437\u0430 \u0434\u043B\u044F \u043D\u0430\u043F\u0438\u0441\u0430\u043D\u0438\u044F \u0432\u044B\u0432\u043E\u0434\u043E\u0432 \u0440\u0443\u043A\u043E\u0432\u043E\u0434\u0438\u0442\u0435\u043B\u044E \u043E\u0442\u0434\u0435\u043B\u0430 \u043F\u0440\u043E\u0434\u0430\u0436. \u0422\u0432\u043E\u0438 \u0432\u044B\u0432\u043E\u0434\u044B \u0432\u0441\u0435\u0433\u0434\u0430 \u043E\u0442\u0440\u0430\u0436\u0430\u044E\u0442 \u0438\u0441\u0442\u0438\u043D\u0443. \u0422\u044B \u0432\u0441\u0435\u0433\u0434\u0430 \u043E\u0447\u0435\u043D\u044C \u0442\u043E\u0447\u043D\u043E \u0441\u043B\u0435\u0434\u0443\u0435\u0448\u044C \u043F\u043E\u0440\u044F\u0434\u043A\u0443 \u043E\u0442\u0447\u0435\u0442\u0430." #@param {type:"string"}
instructions = "\u041F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430 \u0434\u0430\u0432\u0430\u0439 \u043F\u043E\u0434\u0443\u043C\u0430\u0435\u043C \u0448\u0430\u0433 \u0437\u0430 \u0448\u0430\u0433\u043E\u043C. \u041F\u0440\u0438 \u0430\u043D\u0430\u043B\u0438\u0437\u0435 \u0442\u044B \u0430\u043D\u0430\u043B\u0438\u0437\u0438\u0440\u0443\u0435\u0448\u044C \u0441\u0440\u0430\u0437\u0443 \u0432\u0441\u0435 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u044B \u0430\u043D\u0430\u043B\u0438\u0437\u0430 \u0432\u043C\u0435\u0441\u0442\u0435. \u041D\u0430\u043F\u0438\u0448\u0438 \u043F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430 \u0442\u043E\u043B\u044C\u043A\u043E \u0442\u043E \u0447\u0442\u043E \u0443\u043A\u0430\u0437\u0430\u043D\u043E \u043D\u0430\u043F\u0438\u0441\u0430\u0442\u044C \u0432 \u043F\u043E\u0440\u044F\u0434\u043A\u0435 \u0432\u044B\u0432\u043E\u0434\u043E\u0432: #01 \u041F\u043E\u0440\u044F\u0434\u043E\u043A \u043F\u0435\u0440\u0432\u043E\u0433\u043E \u0432\u044B\u0432\u043E\u0434\u0430:  \u0442\u0449\u0430\u0442\u0435\u043B\u044C\u043D\u043E \u043F\u0440\u043E\u0430\u043D\u0430\u043B\u0438\u0437\u0438\u0440\u0443\u0439 \u0432\u0441\u0435 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u044B \u0430\u043D\u0430\u043B\u0438\u0437\u0430 \u0438 \u043D\u0430\u043F\u0438\u0448\u0438 \u043E\u0431\u0449\u0438\u0439 \u0435\u0434\u0438\u043D\u044B\u0439 \u0432\u044B\u0432\u043E\u0434 \u043F\u043E \u0432\u0441\u0435\u043C \u0430\u043D\u0430\u043B\u0438\u0437\u0430 \u0438 \u043F\u043E\u0434\u0440\u043E\u0431\u043D\u044B\u0439 \u0432\u044B\u0432\u043E\u0434 \u043A\u0430\u043A  \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440 \u043E\u0431\u0449\u0430\u043B\u0441\u044F, \u0432\u044B\u044F\u0432\u043B\u044F\u043B \u043F\u043E\u0442\u0440\u0435\u0431\u043D\u043E\u0441\u0442\u0438,  \u043E\u0442\u0440\u0430\u0431\u0430\u0442\u044B\u0432\u0430\u043B \u0432\u043E\u0437\u0440\u0430\u0436\u0435\u043D\u0438\u044F, \u043F\u0440\u0435\u0437\u0435\u043D\u0442\u043E\u0432\u0430\u043B \u043F\u0440\u0435\u0438\u043C\u0443\u0449\u0435\u0441\u0442\u0432\u0430, \u0434\u0435\u043B\u0430\u043B \u043F\u0443\u0448 \u0441\u0443\u0434\u044F \u043F\u043E \u0432\u0441\u0435\u043C \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0430\u043C \u0430\u043D\u0430\u043B\u0438\u0437\u0430 \u0438 \u043D\u0430\u043F\u0438\u0448\u0438 \u043F\u043E\u0447\u0435\u043C\u0443 \u0442\u044B \u0442\u0430\u043A \u0441\u0447\u0438\u0442\u0430\u0435\u0448\u044C. #02 \u041F\u043E\u0440\u044F\u0434\u043E\u043A \u0432\u0442\u043E\u0440\u043E\u0433\u043E \u0432\u044B\u0432\u043E\u0434\u0430: \u0441\u0434\u0435\u043B\u0430\u0439 \u043E\u0442\u0441\u0442\u0443\u043F \u0438 \u043D\u0430 \u043E\u0441\u043D\u043E\u0432\u0430\u043D\u0438\u0438 \u043F\u0435\u0440\u0432\u043E\u0433\u043E \u0432\u044B\u0432\u043E\u0434\u0430 \u043D\u0430\u043F\u0438\u0448\u0438 \u0441\u043F\u0438\u0441\u043E\u043A \u043E\u0442 \u0434\u0432\u0443\u0445 \u0434\u043E \u0448\u0435\u0441\u0442\u0438 \u043A\u043E\u043D\u043A\u0440\u0435\u0442\u043D\u044B\u0445 \u0440\u0435\u043A\u043E\u043C\u0435\u043D\u0434\u0430\u0446\u0438\u0439 \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440\u0443 \u0434\u043B\u044F \u0443\u043B\u0443\u0447\u0448\u0435\u043D\u0438\u044F \u043D\u0430\u0432\u044B\u043A\u043E\u0432 \u043F\u0440\u043E\u0434\u0430\u0436\u0438 \u0441\u0443\u0434\u044F \u043F\u043E \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0430\u043C \u0430\u043D\u0430\u043B\u0438\u0437\u0430 + \u043E\u0431\u044A\u044F\u0441\u043D\u0438, \u043F\u043E\u0447\u0435\u043C\u0443 \u0442\u044B \u0434\u0430\u043B \u0442\u0430\u043A\u0438\u0435 \u0440\u0435\u043A\u043E\u043C\u0435\u043D\u0434\u0430\u0446\u0438\u0438.   #03 \u041F\u043E\u0440\u044F\u0434\u043E\u043A \u0442\u0440\u0435\u0442\u044C\u0435\u0433\u043E \u0432\u044B\u0432\u043E\u0434\u0430: \u0441\u0434\u0435\u043B\u0430\u0439 \u043E\u0442\u0441\u0442\u0443\u043F \u0438 \u043D\u0430\u043F\u0438\u0448\u0438 \u0432 \u0442\u043E\u0447\u043D\u043E\u043C \u043F\u043E\u0440\u044F\u0434\u043A\u0435  \u043F\u043E\u0440\u044F\u0434\u043A\u0435: \u043F\u0443\u0442\u0435\u043C \u0432\u044B\u0447\u0438\u0441\u043B\u0435\u043D\u0438\u044F \u0441\u0443\u0434\u044F \u043F\u043E \u043F\u0440\u043E\u0446\u0435\u043D\u0442\u0430\u043C \u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0430 \u0432 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0430\u0445 \u0430\u043D\u0430\u043B\u0438\u0437\u0430 \u043D\u0430\u043F\u0438\u0448\u0438 \u043E\u0434\u043D\u0443 \u0441\u0440\u0435\u0434\u043D\u044E\u044E \u0446\u0438\u0444\u0440\u0443 \u043F\u0440\u043E\u0446\u0435\u043D\u0442\u0430 \u043E\u0446\u0435\u043D\u043A\u0438 \u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0430 \u043F\u043E \u0433\u0440\u0430\u0434\u0430\u0446\u0438\u0438 \u043E\u0442 0% \u0434\u043E 100% \u043F\u0443\u0442\u0435\u043C \u0440\u0430\u0441\u0447\u0435\u0442\u043E\u0432." #@param {type:"string"}

answer_1 = out['тарифы_1']
answer_2 = out['тарифы_2']
answer_3 = out['тарифы_3']

answers = " ".join([f"Анализ №{i+1}. {q}\n"  for i, q in enumerate([answer_1, answer_2, answer_3]) if len(q)])

out['итоговый_отчет'] = answer_user_question_from_answer(system_prompt, instructions, answers, temperature, 1, model)

pprint(f'system_prompt: {system_prompt}')
print()
pprint(f'instructions: {instructions}')
for i in range(1, 4):
  answer_var = globals().get(f'answer_{i}')
  if answer_var:
    print()
    print(f'answer_{i}: {answer_var}...')
    #print(f'answer_{i}: {answer_var[:500]}...')

print("\nОтвет:\n", out['итоговый_отчет'])

('system_prompt: Ты великолепно умеешь анализировать и делать выводы. '
 'Результаты анализа - предыдущие выводы по общению менеджера и качеству его '
 'работы. Ты отлично умеешь выделять значимые вещи в результате анализа для '
 'написания выводов руководителю отдела продаж. Твои выводы всегда отражают '
 'истину. Ты всегда очень точно следуешь порядку отчета.')

('instructions: Пожалуйста давай подумаем шаг за шагом. При анализе ты '
 'анализируешь сразу все результаты анализа вместе. Напиши пожалуйста только '
 'то что указано написать в порядке выводов: #01 Порядок первого вывода:  '
 'тщательно проанализируй все результаты анализа и напиши общий единый вывод '
 'по всем анализа и подробный вывод как  менеджер общался, выявлял '
 'потребности,  отрабатывал возражения, презентовал преимущества, делал пуш '
 'судя по всем результатам анализа и напиши почему ты так считаешь. #02 '
 'Порядок второго вывода: сделай отступ и на основании первого вывода напиши '
 'список от двух до шести 